In [1]:
import random

def generar_matriz_mutada(matriz_original, num_gaps):
    matriz_mutada = []
    

    for fila in matriz_original:
        fila_mutada = []
        for caracter in fila:                                                                                                            
            fila_mutada.append(f'[{caracter}]')  #Encerrar el caracter original entre corchetes para un mejor orden
            if random.random() < 0.5:  # Insertar un gap con probabilidad asignada
                fila_mutada.append('[-]')  # Insertar "-" entre caracteres encerrados para no generarse fuera de ellos

        # Agregar gaps adicionales según la probabilidad 
        for _ in range(num_gaps):
            fila_mutada.insert(random.randint(0, len(fila_mutada)), '[-]')
        matriz_mutada.append(fila_mutada)

    return matriz_mutada

def cuadriplicar_matriz(matriz_original, num_gaps):
    matrices_cuadriplicadas = []

    for _ in range(4):  # Cuadriplicar la matriz original o determinar cuantas veces duplicarla dependiendo del rango deseado
        matriz_mutada = generar_matriz_mutada(matriz_original, num_gaps)
        matrices_cuadriplicadas.append(matriz_mutada)

    return matrices_cuadriplicadas

def imprimir_matriz_con_puntaje(matriz, puntaje):
    for i, matriz_mutada in enumerate(matriz):
        print(f"Mutación {i + 1} (Puntaje: {puntaje[i]} puntos):")
        for fila_mutada in matriz_mutada:
            fila_str = ''.join(caracter for caracter in fila_mutada if caracter not in '[ ]')  # Unir los caracteres excluyendo '[ ]'
            fila_str = fila_str.replace('[ ]', '-')  # Reemplazar '[ ]' por '-'
            print(fila_str)

def imprimir_matriz(matriz):
    for fila in matriz:
        fila_str = ''.join(fila)
        print(fila_str)

def calcular_puntaje(matriz):
    puntaje_total = 0

    if not matriz or not matriz[0]:  # Verificar si la matriz está vacía o sin filas
        return puntaje_total

    n_filas = len(matriz)
    n_columnas = len(matriz[0])
    
    for col in range(n_columnas):
        conteo_letras = {}
        for fila in matriz:
            if col < len(fila):  # Verificar si la columna es válida para esta fila
                caracter = fila[col]
                if caracter != '[ ]' and caracter != '[-]':
                    if caracter not in conteo_letras:
                        conteo_letras[caracter] = 1
                    else:
                        conteo_letras[caracter] += 1
        
        for letra, repeticiones in conteo_letras.items():
            if repeticiones > 1:
                puntaje_total += 20 * (repeticiones - 1)  # Sumar puntos por repeticiones
        
    return puntaje_total

def seleccionar_mejores_mutaciones(matrices, n_mejores):
    puntajes = []

    for i, matriz_mutada in enumerate(matrices):
        puntaje = calcular_puntaje(matriz_mutada)
        puntajes.append((i, puntaje))  # Guardar el índice de la mutación y su puntaje
    
    # Ordenar mutaciones por puntaje descendente
    puntajes.sort(key=lambda x: x[1], reverse=True)

    # Seleccionar las mejores mutaciones y sus puntajes
    mejores_mutaciones = [matrices[idx] for idx, _ in puntajes[:n_mejores]]
    mejores_puntajes = [puntaje for _, puntaje in puntajes[:n_mejores]]

    return mejores_mutaciones, mejores_puntajes

def partir_matriz_en_partes(matriz, num_partes):
    if not matriz:
        return []

    filas = len(matriz)
    columnas = len(matriz[0])

    if num_partes <= 0:
        raise ValueError("El número de partes debe ser mayor que cero.")

    # Determinar el número de filas en cada parte
    filas_por_parte = filas // num_partes
    resto_filas = filas % num_partes  # Filas restantes después de dividir en partes iguales

    partes = []

    inicio_fila = 0
    for i in range(num_partes):
        alto_actual = filas_por_parte
        # Ajustar la altura de la parte para distribuir las filas restantes
        if resto_filas > 0:
            alto_actual += 1
            resto_filas -= 1

        fin_fila = inicio_fila + alto_actual

        # Extraer las filas correspondientes para esta parte
        parte = matriz[inicio_fila:fin_fila]
        partes.append(parte)

        # Actualizar el inicio para la próxima parte
        inicio_fila = fin_fila

    return partes

def obtener_secuencias():
    secuencias = []
    num_secuencias = int(input("¿Cuántas secuencias (palabras) deseas agregar? "))

    if num_secuencias <= 0:
        raise ValueError("El número de secuencias debe ser mayor que cero.")

    for i in range(num_secuencias):
        palabra = input(f"Introduce la palabra para la secuencia {i + 1}: ")
        secuencias.append(palabra)

    return secuencias

def generar_hijos(mutacion_padre, mutacion_madre):
    # Obtener las partes de la mutación padre y madre
    partes_padre = partir_matriz_en_partes(mutacion_padre, len(mutacion_padre))  # Dividir en partes igual al número de filas
    partes_madre = partir_matriz_en_partes(mutacion_madre, len(mutacion_madre))  # Dividir en partes igual al número de filas

    hijos = []

    # Generar dos hijos combinando partes de manera alternada
    for i in range(2):  # Generar 2 hijos
        hijo = []
        for j in range(len(partes_padre)):  # Combinar todas las partes para cada hijo
            if random.random() < 0.5:  # Probabilidad del 50% de tomar parte del padre o la madre
                parte = partes_padre[j]
            else:
                parte = partes_madre[j]
            
            hijo.extend(parte)  # Extender el hijo con la parte actual
        
        hijos.append(hijo)

    return hijos

def limpiar_mutacion(mutacion):
    # Obtener dimensiones de la mutación
    if not mutacion or not mutacion[0]:
        return mutacion

    n_filas = len(mutacion)
    n_columnas = len(mutacion[0])

    # Determinar qué columnas consisten completamente de gaps
    columnas_a_eliminar = set()
    for col in range(n_columnas):
        columna_completa_gap = True
        for fila in mutacion:
            if col < len(fila) and fila[col] != '[-]':
                columna_completa_gap = False
                break
        if columna_completa_gap:
            columnas_a_eliminar.add(col)

    # Crear una nueva mutación sin las columnas de gaps completos
    mutacion_limpiada = []
    for fila in mutacion:
        fila_limpiada = [fila[col] for col in range(n_columnas) if col not in columnas_a_eliminar and col < len(fila)]
        mutacion_limpiada.append(fila_limpiada)

    return mutacion_limpiada

def main():
    secuencias = obtener_secuencias()
    
    matriz_original = []
    
    for seq in secuencias:
        fila = list(seq)  # Convertir la secuencia en una lista de caracteres
        matriz_original.append(fila)
    
    print("\nMatriz Original:")
    imprimir_matriz(matriz_original)
    
    num_generaciones = 20  # Número de generaciones a ejecutar
    num_gaps = 1  # Número inicial de gaps

    matrices_cuadriplicadas = cuadriplicar_matriz(matriz_original, num_gaps)[:4]  # Generar 4 mutaciones iniciales

    for generacion in range(num_generaciones):
        print(f"\nGeneración {generacion + 1}:")
        
        # Seleccionar las mejores 2 mutaciones según el puntaje de la generación actual
        mejores_mutaciones, _ = seleccionar_mejores_mutaciones(matrices_cuadriplicadas, 2)
        mejor_1 = mejores_mutaciones[0]
        mejor_2 = mejores_mutaciones[1]
        
        # Generar hijos a partir de las partes de las mejores mutaciones seleccionadas (mejor_1 y mejor_2)
        hijos = generar_hijos(mejor_1, mejor_2)

        # Aumentar el número de gaps para la próxima generación
        num_gaps += 1
        
        # Generar nuevas mutaciones con el nuevo número de gaps
        matrices_cuadriplicadas = cuadriplicar_matriz(matriz_original, num_gaps)[:4]  # Generar 4 nuevas mutaciones
        
        # Combinar todas las matrices (mejor_1, mejor_2, hijos, nuevas_mutaciones) para la siguiente generación
        matrices_cuadriplicadas = [mejor_1, mejor_2] + hijos + matrices_cuadriplicadas
        
        # Imprimir las matrices y puntajes de la generación actual
        imprimir_matriz_con_puntaje(matrices_cuadriplicadas, [calcular_puntaje(m) for m in matrices_cuadriplicadas])

    # Tomar la última mutación generada y limpiar las columnas de gaps completos
    ultima_mutacion = matrices_cuadriplicadas[-1]
    mutacion_limpiada = limpiar_mutacion(ultima_mutacion)

    print("\nÚltima Mutación (Después de limpiar columnas de gaps):")
    imprimir_matriz(mutacion_limpiada)

if __name__ == "__main__":
    main()


Matriz Original:
EXPLORAR
DETERMINACION
SOLIDARIDAD

Generación 1:
Mutación 1 (Puntaje: 60 puntos):
[-][E][-][X][-][P][L][O][-][R][-][A][-][R][-]
[-][D][-][E][T][E][R][-][M][I][N][A][C][-][I][O][-][N][-]
[S][O][L][I][D][A][-][-][R][I][D][A][D][-]
Mutación 2 (Puntaje: 40 puntos):
[E][X][-][-][P][-][L][-][O][R][-][A][-][R]
[D][E][-][T][E][-][R][-][M][I][N][A][C][I][-][-][O][-][N][-]
[S][-][O][-][L][-][-][I][D][-][A][R][-][I][D][A][D]
Mutación 3 (Puntaje: 20 puntos):
[-][E][-][X][-][P][L][O][-][R][-][A][-][R][-]
[-][D][-][E][T][E][R][-][M][I][N][A][C][-][I][O][-][N][-]
[S][-][O][-][L][-][-][I][D][-][A][R][-][I][D][A][D]
Mutación 4 (Puntaje: 20 puntos):
[E][X][-][-][P][-][L][-][O][R][-][A][-][R]
[-][D][-][E][T][E][R][-][M][I][N][A][C][-][I][O][-][N][-]
[S][-][O][-][L][-][-][I][D][-][A][R][-][I][D][A][D]
Mutación 5 (Puntaje: 40 puntos):
[E][-][X][-][P][L][-][O][R][A][-][R][-]
[D][E][T][E][R][-][M][-][I][-][N][-][A][C][I][-][O][-][N][-]
[S][O][-][L][-][I][-][-][D][A][-][R][-][I][-][D][-][-]